# Notebook: Gráficos para o Streamlit
Este notebook tem por objetivo criar a base dos gráficos apresentados pelo projeto no Streamlit.

# 1) Imports

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from scipy.stats import gaussian_kde
import numpy as np

# 2) Carregamento das bases de dados

In [ ]:
df_2020 = pd.read_csv('processado_base_2020.csv', sep=';')
df_2021 = pd.read_csv('processado_base_2021.csv', sep=';')
df_2022 = pd.read_csv('processado_base_2022.csv', sep=';')
df = pd.read_csv('processado_base_full.csv', sep=';')

# 3) Funções para output dos gráficos

In [ ]:
def plot_bar(df, col, titulo, xaxis, yaxis='Quantidade'):
  grupos = df[col].value_counts()

  fig = go.Figure(
      go.Bar(
          x=grupos.index,
          y=grupos,
          text=grupos,
          textposition='auto'
      )
  )

  fig.update_layout(
      title=titulo,
      xaxis=dict(tickmode='linear'),
      xaxis_title=xaxis,
      yaxis_title=yaxis,
  )

  fig.show()

In [ ]:
def plot_histograma(df, col, titulo, rug=True):
  # faz o cálculo do KDE com o scipy
  data = df[col].values
  kde = gaussian_kde(data)
  x_vals = np.linspace(min(data), max(data), 1000)
  kde_vals = kde(x_vals)

  # faz o cálculo da quantidade ótima de bins (assim evitamos agrupamentos desnecessários)
  bins = len(np.histogram_bin_edges(data, bins='auto'))

  # cria os plots separados (histograma + kde + rug)
  # 1. histograma
  histogram = go.Histogram(x=data, nbinsx=bins, histnorm='probability density', name=f'Densidade: {col}')

  # 2. kde
  kde_line = go.Scatter(x=x_vals, y=kde_vals, mode='lines', name='Curva (KDE)', line=dict(color='red'))

  # 3. rug, mas apenas se ele tiver sido requisitado
  if rug:
    rug_plot = go.Scatter(
        x=data,
        y=[-0.01] * len(data),
        mode='markers',
        name='Observações individuais',
        marker=dict(color='black', symbol='line-ns-open', size=10)
    )

  # figura principal
  fig = go.Figure()
  fig.add_trace(histogram)
  fig.add_trace(kde_line)
  fig.update_traces(texttemplate='%{y:.2%}', textposition='outside', selector=dict(type='histogram'))

  # configs
  fig.update_layout(
      title=titulo,
      xaxis_title='Valor',
      yaxis_title='Frequência',
      yaxis=dict(range=[0, max(kde_vals) + 0.1]),
      bargap=0.015,
      uniformtext_mode='hide'
  )

  # configs com rug
  if rug:
    fig.add_trace(rug_plot)
    fig.update_layout(yaxis=dict(range=[-0.02, max(kde_vals) + 0.1]))
  # configs sem rug
  else:
    fig.update_layout(xaxis=dict(tickmode='linear'))

  fig.show()

In [ ]:
  def plot_histograma_comparativo(col):
    fig = px.histogram(data_frame=df, x=col, color='ANO', title=f'Distribuição do indicador {col} comparativo', nbins=50, marginal='rug', histnorm='probability density')

    # TODO: não ficou legal... se der tempo, melhorar esse KDE
    # for ano in df['ANO'].unique():
    #     data_year = df[df['ANO'] == ano][col]

    #     kde = gaussian_kde(data_year)
    #     x_vals = np.linspace(min(data_year), max(data_year), 1000)
    #     kde_vals = kde(x_vals)

    #     fig.add_trace(go.Scatter(x=x_vals, y=kde_vals, mode='lines', name=f'KDE {ano}'))

    fig.update_layout(
        xaxis_title='Valor',
        yaxis_title='Frequência',
        height=600
    )

    fig.show()

In [ ]:
def plot_boxplot(df, col, titulo):
  fig = px.box(y=df[col], points="all", title=titulo)

  fig.update_layout(
      yaxis_title='Valor'
  )

  fig.update_yaxes(dtick=1)

  fig.show()

In [ ]:
def plot_boxplot_comparativo(df, col):
  fig = px.box(data_frame=df, x='ANO', y=col, points="all", title=f'Distribuição do {col} comparativa', color='ANO')

  fig.update_layout(
      yaxis_title='Valor'
  )

  fig.update_yaxes(dtick=1)

  fig.show()

In [ ]:
def plot_boxplot_analise_indicador(col):
  plot_boxplot(df_2020, col, f'Distribuição do {col} para o ano de 2020')
  plot_boxplot(df_2021, col, f'Distribuição do {col} para o ano de 2021')
  plot_boxplot(df_2022, col, f'Distribuição do {col} para o ano de 2022')
  plot_boxplot_comparativo(df, col)

# 4) Análise e criação dos gráficos
Nas próximas seções, vamos explorar a base de dados com diferentes tipos de análises e visualizações usando o Plotly. Escolhemos o Plotly por sua alta interatividade, especialmente quando integrado à plataforma Streamlit, proporcionando uma experiência mais rica e dinâmica.

## 4.1) Idade

In [ ]:
def processar_dist_idade(df, ano):
  df['IDADE_ALUNO'] = pd.to_numeric(df['IDADE_ALUNO'], errors='coerce')
  display(df['IDADE_ALUNO'].value_counts().sort_index())
  display(df['IDADE_ALUNO'].describe())
  plot_bar(df, 'IDADE_ALUNO', f'Idades dos alunos em {ano}', xaxis='Idade')
  plot_histograma(df, 'IDADE_ALUNO', f'Distribuição da idade dos alunos em {ano}', rug=False)
  plot_boxplot(df, 'IDADE_ALUNO', f'Boxplot das idades dos alunos em {ano}')

In [ ]:
processar_dist_idade(df_2020, 2020)

IDADE_ALUNO
7       9
8      43
9      59
10    107
11     93
12     86
13     80
14     72
15     50
16     54
17     45
18     21
19      7
20      1
Name: count, dtype: int64

count    727.000000
mean      12.408528
std        2.829807
min        7.000000
25%       10.000000
50%       12.000000
75%       14.000000
max       20.000000
Name: IDADE_ALUNO, dtype: float64

In [ ]:
df_2021

,NOME,FASE,TURMA,INSTITUICAO_ENSINO_ALUNO,SINALIZADOR_INGRESSANTE,PEDRA,INDE,IAA,IEG,IPS,...,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,PONTO_VIRADA,IPV,IAN,NIVEL_IDEAL,DEFASAGEM,ANO
0,ALUNO-1,2.0,G,Rede Decisão,Ingressante,Ametista,7.319,7.5,6.3,6.9,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,8.1,5.0,Nível 3 (7o e 8o ano),-1.0,2021
1,ALUNO-3,3.0,F,Escola Pública,Veterano,Ametista,7.021,7.1,6.9,6.3,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,7.2,10.0,Nível 3 (7o e 8o ano),0.0,2021
2,ALUNO-5,2.0,L,Escola Pública,Veterano,Ametista,7.399,7.4,7.5,7.5,...,Promovido de Fase,Mantido na Fase atual,Mantido na Fase atual,Não avaliado,Não,7.4,10.0,Nível 2 (5o e 6o ano),0.0,2021
3,ALUNO-8,5.0,L,Rede Decisão,Veterano,Topázio,8.269,8.8,8.6,7.5,...,Promovido de Fase + Bolsa,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não,8.5,10.0,Nível 5 (1o EM),0.0,2021
4,ALUNO-11,1.0,A,Rede Decisão,Veterano,Topázio,8.990,9.0,10.0,6.3,...,Promovido de Fase,Mantido na Fase atual,Mantido na Fase atual,Não avaliado,Sim,9.7,10.0,Nível 1 (4o ano),0.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,ALUNO-1338,2.0,C,Escola Pública,Veterano,Ágata,6.511,7.9,2.1,7.5,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,8.7,10.0,Nível 2 (5o e 6o ano),0.0,2021
680,ALUNO-1343,2.0,K,Escola Pública,Veterano,Quartzo,3.996,10.0,1.3,5.6,...,Mantido na Fase atual,Não avaliado,Mantido na Fase atual,Não avaliado,Não,3.7,5.0,Nível 3 (7o e 8o ano),-1.0,2021
681,ALUNO-1344,2.0,R,Escola Pública,Veterano,Ágata,5.664,9.5,5.8,5.6,...,Mantido na Fase atual,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,7.6,2.5,Nível 5 (1o EM),-3.0,2021
682,ALUNO-1345,0.0,L,Escola Pública,Ingressante,Ágata,6.606,9.0,6.7,7.5,...,Mantido na Fase atual,Mantido na Fase atual,Não avaliado,Não avaliado,Não,7.1,5.0,Nível 1 (4o ano),-1.0,2021


In [ ]:
# não há idade dos alunos em 2021 e 2022 :(... mas e se fosse calculado? se der tempo, fazer o cálculo...
#processar_dist_idade(df_2021, 2021)

## 4.2) Instituição de ensino

In [ ]:
plot_bar(df_2020, 'INSTITUICAO_ENSINO_ALUNO', 'Instituições de ensino dos alunos em 2020', xaxis='Instituição de ensino')

In [ ]:
plot_bar(df_2021, 'INSTITUICAO_ENSINO_ALUNO', 'Instituições de ensino dos alunos em 2021', xaxis='Instituição de ensino')

## 4.3) Indicadores

### 4.3.1) INDE

In [ ]:
plot_histograma(df_2020, 'INDE', 2020)

In [ ]:
plot_histograma(df_2021, 'INDE', 2021)

In [ ]:
plot_histograma(df_2022, 'INDE', 2022)

In [ ]:
plot_histograma_comparativo('INDE')

In [ ]:
plot_boxplot_analise_indicador('INDE')

### 4.3.2) IAN

In [ ]:
plot_boxplot_analise_indicador('IAN')

### 4.3.3) IAA

In [ ]:
plot_boxplot_analise_indicador('IAA')

### 4.3.4) IEG

In [ ]:
plot_boxplot_analise_indicador('IEG')

### 4.3.5) IPS

In [ ]:
plot_boxplot_analise_indicador('IPS')

### 4.3.6) IDA

In [ ]:
plot_boxplot_analise_indicador('IDA')

### 4.3.7) IPP

In [ ]:
plot_boxplot_analise_indicador('IPP')

### 4.3.8) IPV

In [ ]:
plot_boxplot_analise_indicador('IPV')